# 深度学习相关要点——权重初始化（待更新）

作者：杨岱川

时间：2020年01月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

参考文献：

- 《深度学习入门》，作者：斋藤康毅；
- 《深度学习》，作者：Ian Goodfellow 、Yoshua Bengio、Aaron Courville。
- [CS231n](http://cs231n.stanford.edu/)

## 本节目的

在神经网络的学习中，权重的初始值特别重要。

实际上，设定什么样的权重初始值，经常关系到神经网络的学习能否成功。本节将介绍权重初始值的推荐值，并通过实验的方式来确认神经网络的学习能否快速进行。

## 如果将权重初始值设为0？

在后续，我们会介绍抑制过拟合、提高泛化能力的技巧——**权值衰减（weight decay）**。

简单地说，权值衰减就是一种以减小权重参数的值为目的的学习方法。通过减小权重参数的值来抑制过拟合的发生。

--------------

如果想减小权重的值，一开始就将初始值设为较小的值才是正道。实际上，之前的权重初始值都是像 `0.01 * np.random.ran(10, 100)` 这样，使用由**高斯分布**生成的值乘以 0.01 后得到的值，即标准差为 0.01 的高斯分布。

如果我们把权重初始值全部设为 0 以减小权重的值，会怎么样呢？从结论上讲，这不是一个好选择。

> 事实上，将权重初始值设为 0 ，神经网络就没法正确学习了。

为什么不能将权重初始值设置为 0 呢？更严格的说法是，为什么不能将权重初始值设为**一样**的值呢？

这是因为在误差反向传播法中，所有的权重值都会进行相同的更新。

先回忆 SGD ：

$$\large \textbf{W}\leftarrow\textbf{W}-\eta\frac{\partial L}{\partial\textbf{W}}$$

比如在 2 层神经网络中，假设第 1 层和第 2 层权重为0，这样一来，正向传播中，由于输入层的权重为 0 ，所以第 2 层的神经元会全部被传递相同的值。

第 2 层的神经元中全部输入相同的值，这意味着反向传播时第 2 层的权重就会进行相同的更新。

因此，权重被更新为相同的值，并且拥有了重复的值。这使得神经网络拥有许多不同权重的意义消失了。所以我们必须**随机生成初始值**。

## 不同初始值的比较

我们希望尝试一下不同分布的初始值的影响。

In [1]:
import argparse
import os

import matplotlib.pyplot as plt 
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm_notebook

class Net(nn.Module):
    """定义神经网络Net"""
    def __init__(self):
        """初始化各个层"""
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=28*28, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        """定义前向传播"""
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [2]:
torch.__version__

'1.4.0'